### Tr for Single Angle Bolted Through One Leg

Calculate factored tension resistance, $T_r$, of a single angle bolted through one leg.

![Angle bolted through one leg](images/bolted-single-angle.svg)

_Note:_ The figure shows 2 rows of 4 bolts through one leg.  The logic below works for
1 or 2 rows and almost any number of bolts per row.

In [28]:
from Designer import Designer as DesignNotes, SST, show
from newparts import Part

In [2]:
Table6 = {'M16': (28,22),    # CSA S16-14 Table 6  - Minimum edge distances
          '3/4': (32,25),
          'M20': (34,26),
          '7/8': (38,28),
          'M22': (38,28),
          'M24': (42,30),
          '1':   (44,32),
         }

def get_Table6(size,col=1):
    assert col in [1,2], "Must select column 1 or 2 from Table 6."
    try:
        return Table6[size][col-1]
    except KeyError:
        pass
    raise KeyError('Cannot determine min edge distance for bolt size: "{}"'.format(size))

In [4]:
BOLT_DIA = {'M16': 16.,
           '3/4': 25.4*3/4,
           'M20': 20.,
           '7/8': 25.4*7/8,
           'M22': 22.,
           'M24': 24.,
           '1': 25.4,
           }

HOLE_DIA = {'M16': 18.,
           '3/4': 22.,
           'M20': 22.,
           '7/8': 24.,
           'M22': 24.,
           'M24': 26.,
           '1': 27.,
           }

BOLT_Fu = {'A325M': 830.,
          'A490M': 1040.,
          'A325': 825.,
          'A490': 1035.,
          }

In [36]:
#           Leg:   g, g1
UG_Angle = {203: (115,75),    # Usual Gauges for Angles, CISC Handbook 11th ed., p. 6-173
           178: (100,65),
           152: (90,60),
           127: (75,50),
           102: (65,),
           89: (50,),
           76: (45,),
           64: (35,),
           51: (29,),
           44: (25,),
            }

# select largest g2 from 45,50,55,60,65,70,75 > 2.7d and edge dist ok

def get_UG_Angle(leg,nlines=1,size='3/4'):
    leg = int(leg)
    if leg not in UG_Angle:
        raise ValueError('Cannot determine gauges for angle leg of size "{}"'.format(leg))
    
    gs = UG_Angle[leg]
    if nlines == 1:
        return gs[0],0
    if len(gs) == 1:
        raise ValueError('Leg size of "{}" cannot have 2 rows of bolts'.format(leg))
    g1 = gs[1]
    min_edge = get_Table6(size,col=2)
    d = BOLT_DIA[size]
    gmin = 2.7*d                         # CSA S16-14 23.3.1
    for g2 in [80,75,70,65,60,55,50,45]:
        if g2 >= gmin and (leg-g1-g2) >= min_edge:
            return g1,g2
    raise ValueError('Unable to determine workable gauges for angle, leg={}, size={}'.format(leg,size))

In [30]:
Bolt = Part( 'Bolts',
              type = 'A325',
              size = '3/4',
              hole_type = 'punched',  # 'punched' or 'drilled'
              Nrows = 1,              # a row is parallel to the load
              Nlines = 4,             # a line is perpendicular to the load
              pitch = 75.,            # spacing measure paprallel to load (along length of angle)
              end_distance = 30.,     # from end of angle to center of closest hole
              )

Bolt.set( Fu = BOLT_Fu[Bolt.type],
           d = BOLT_DIA[Bolt.size],
           ha = HOLE_DIA[Bolt.size] + (0. if Bolt.hole_type == 'drilled' else 2.),
           )
Bolt.vars()

{'Fu': 825.0,
 'Nlines': 4,
 'Nrows': 1,
 '_doc': 'Bolts',
 'd': 19.049999999999997,
 'end_distance': 30.0,
 'ha': 24.0,
 'hole_type': 'punched',
 'pitch': 75.0,
 'size': '3/4',
 'type': 'A325'}

In [31]:
Dsg = 'L102x76x13'
Ag,d,b,t = SST.section(Dsg,'A,D,B,T')
Angle = Part( 'Angle',
               Fy = 350.,
               Fu = 450.,
               dsg = Dsg,
               Ag = Ag,
               d = d,
               b = b,
               t = t,
               bolted_leg = 'long',   # 'short' or 'long'
               m = 1,    # number of faying surfaces
               threads_intercepted = True)
Angle.vars()

{'Ag': 2100.0,
 'Fu': 450.0,
 'Fy': 350.0,
 '_doc': 'Angle',
 'b': 76.2,
 'bolted_leg': 'long',
 'd': 102.0,
 'dsg': 'L102x76x13',
 'm': 1,
 't': 12.7,
 'threads_intercepted': True}

In [32]:
des = DesignNotes(var='Tr',units='kN',trace=True)

REQ = des.require   # useful abbreviations
CHK = des.check       
REC = des.record

### Sanity check of input data:

In [33]:
REQ(Bolt.size in BOLT_DIA,'bolt size is not one of available sizes',
    size=Bolt.size, SIZES=BOLT_DIA.keys())
REQ(Bolt.hole_type in ['punched','drilled'], "Value of 'hole_type' must be punched or drilled")
REQ(Angle.threads_intercepted in [False,True],"Value of 'threads_intercepted' must be True or False")
REQ(Angle.m in [1,2], 'Number of faying surfaces must be 1 or 2')
REQ(Angle.bolted_leg in ['short','long'],"Value of 'bolted_leg' must be short or long")
REQ(Bolt.Nrows in [1,2],"Nrows must be 1 or 2")
REQ(Bolt.Nlines >= 2 and Bolt.Nlines <= 10,"Nlines must be in range of 2 to 10")

In [37]:
# data derived from input data:  pitch, end_distance, gauges, etc.

Ag,d,b,t,bleg = Angle['Ag,d,b,t,bolted_leg']
if bleg == 'short':  # bolts go thru leg of size 'd'
    d,b = b,d
    
bolt_diameter,pitch = Bolt['d,pitch']
    
min_end_distance = min_edge_distance = get_Table6(Bolt.size,col=1)  # CSA S16-14 22.3.3
if Bolt.Nlines <= 2:
    min_end_distance = 1.5*bolt_diameter  # CSA S16-01 22.3.4

g1,g2 = get_UG_Angle(d,Bolt.Nrows,Bolt.size)
edge_distance = d-(g1+g2)
show('Ag,d,b,t,g1,g2,edge_distance')

Ag            = 2100
d             = 102
b             = 76.2
t             = 12.7
g1            = 65
g2            = 0
edge_distance = 37


In [34]:
raise Exception('Converted to Here!')

Exception: Converted to Here!

### Check Bolting Details
Failure to meet requirements is not a fatal error. Results are reported in the work record.

In [ ]:
# CSA S16-09  22.3.1 thru .4:
max_edge_distance = min(12.*t,150.)
REQ(edge_distance > bolt_diameter/2.,'Angle leg of {0} mm does not support {1} lines of bolts.'.format(d,Nlines))

CHK(pitch >= 2.7*bolt_diameter,'Pitch','pitch',min_pitch=2.7*bolt_diameter)
CHK(edge_distance >= min_edge_distance,'Edge distance','edge_distance,min_edge_distance')
CHK(edge_distance <= max_edge_distance,'Edge distance','edge_distance,max_edge_distance')
CHK(end_distance >= min_end_distance,'End distance','end_distance,min_end_distance')

# CSA S16-09  28.4.1:
hole_allowance = hole_diameter = bolt_diameter + 2.
if hole_type == 'punched':
    CHK(t < bolt_diameter+4,'Punched holes','bolt_diameter,t',max_t=bolt_diameter+4)

# CSA S16-09  12.3.2:
if hole_type == 'punched':
    hole_allowance += 2.

### Strength Calculations

In [ ]:
# CSA S16-09  13.1
phi = 0.90
phiu = 0.75
phib = 0.80 
phibr = 0.80

#### Gross section yield:

In [ ]:
# CSA S16-09   13.2 (a) (i):
REC('Gross area yield','Ag',Tr=phi*Ag*Fy/1000.)

#### Net section fracture:

In [ ]:
# CSA S16-09   13.2 (a) (iii):
An = Ag - Nlines*hole_allowance*t
# CSA S16-09   12.3.3.2 (b)
if Nrows >= 4:
    Ane = 0.80*An
else:
    Ane = 0.60*An
REC('Net area fracture','An,Ane',Tr=phiu*Ane*Fu/1000.)

#### Block shear failure:
![Block Shear Patterns](bolted-single-angle-shear-blocks.svg)

In [ ]:
#  CSA S16-09    13.11
e = end_distance
L = (Nrows-1.)*pitch

# Case 1 - one shear area, tension failure from furthest bolt to edge
An = (d - g1 - (Nlines-0.5)*hole_allowance)*t
Agv = (e+L)*t
Ut = 0.6
Fv = (Fy+Fu)/2.
if Fy > 485:     # CSA S16-09   13.11  (foot note)
    Fv = Fy
REC('Block shear (case 1)','An,Agv,Ut',Tr=phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.)

In [ ]:
# Case 2 - for 2 or more lines of bolts
if Nlines > 1:
    An = (g2 - hole_allowance)*t
    Agv = 2.*(e+L)*t
    Ut = 0.6
    REC('Block shear (case 2)','An,Agv,Ut',Tr=phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.)

In [ ]:
# Case 3 - tearout
An = 0.
Agv = 2.*(e+L)*t*Nlines
Ut = 0.
REC('Block shear (tearout)','An,Agv,Ut',Tr=phiu*(Ut*An*Fu + 0.6*Agv*Fv)/1000.)

#### Fastener strength, bearing-type connection: bolts in shear

In [ ]:
m = 1
if shear_type == 'double':
    m = 2
n = Nrows*Nlines
db = bolt_diameter
L = (Nrows-1.)*pitch   # length of connection

# CSA S16-09   13.12.1.2 (c)
multiplier = 1.0
if L >= 15.*d:
    multiplier = max(0.75,1.075 - 0.005*L/d)
if threads_intercepted:
    multiplier *= 0.70
Vr = multiplier*0.6*phib*n*m*(3.1415926*db*db/4.)*Fub/1000.
REC('Bolt Shear',multiplier=multiplier,n=n,Tr=Vr)

#### Bolts in bearing:

In [ ]:
# CSA S16-09    13.12.1.2:
Br = 3.*phibr*n*t*db*Fu/1000.
REC('Bolt Bearing',n=n,d=db,t=t,Tr=Br)

#### Combined tearout and bearing:

In [ ]:
# combine tearout of bolts closest to end with bearing of remainder
e = end_distance
Agv = 2.*e*t*Nlines
n = Nlines*(Nrows-1)
Tr = phiu*(0.6*Agv*Fv)/1000.
Br = 3.*phibr*n*t*db*Fu/1000.
REC('Bolt bearing + end tearout',n=n,t=t,d=db,Agv=Agv,Tr=Tr+Br)

### Summary:

In [ ]:
des.show('Fy,Fu,AngleDsg,Ag,d,b,t,Fub,bolt_size,bolt_diameter,hole_type',
         'pitch,end_distance,threads_intercepted,shear_type,bolted_leg,Nlines,Nrows',
         'end_distance,edge_distance,g1,g2')
des.summary()          